## Crear dataset

In [1]:
import numpy as np
import pandas as pd

# Configuración del brazo robótico
l1, l2, l3 = 1, 1, 1  # Longitudes de los segmentos del brazo

# Función para calcular la posición (x, y) del extremo del brazo dado los ángulos
def forward_kinematics(theta1, theta2, theta3):
    x = l1 * np.cos(theta1) + l2 * np.cos(theta1 + theta2) + l3 * np.cos(theta1 + theta2 + theta3)
    y = l1 * np.sin(theta1) + l2 * np.sin(theta1 + theta2) + l3 * np.sin(theta1 + theta2 + theta3)
    return x, y

# Generar dataset
num_samples = 1000
theta1 = np.random.uniform(0, 2 * np.pi, num_samples)
theta2 = np.random.uniform(0, 2 * np.pi, num_samples)
theta3 = np.random.uniform(0, 2 * np.pi, num_samples)

data = []
for t1, t2, t3 in zip(theta1, theta2, theta3):
    x, y = forward_kinematics(t1, t2, t3)
    data.append([t1, t2, t3, x, y])

df = pd.DataFrame(data, columns=['theta1', 'theta2', 'theta3', 'x', 'y'])
df.to_csv('dataset/robot_arm_dataset.csv', index=False)

## Crear RN y entrenar

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam

# Cargar el dataset
df = pd.read_csv('dataset/robot_arm_dataset.csv')

# Normalizar los datos
scaler = StandardScaler()
X = scaler.fit_transform(df[['theta1', 'theta2', 'theta3']].values)
y = df[['x', 'y']].values

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de la red neuronal con mayor complejidad
model = Sequential([
    Dense(128, input_dim=3, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='linear')
])

# Compilar el modelo con una tasa de aprendizaje menor
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=16)



c:\Users\uriel\OneDrive\Escritorio\Uriel\Python\RoboticArm\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.5350 - loss: 1.4607 - val_accuracy: 0.7312 - val_loss: 1.0436
Epoch 2/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6747 - loss: 0.9702 - val_accuracy: 0.7625 - val_loss: 0.7820
Epoch 3/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7388 - loss: 0.8163 - val_accuracy: 0.7625 - val_loss: 0.6262
Epoch 4/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7953 - loss: 0.5646 - val_accuracy: 0.8062 - val_loss: 0.4864
Epoch 5/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8351 - loss: 0.4740 - val_accuracy: 0.8125 - val_loss: 0.3796
Epoch 6/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8226 - loss: 0.3853 - val_accuracy: 0.8125 - val_loss: 0.3267
Epoch 7/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8632 - loss: 0.3045 - val_accuracy: 0.8438 - val_loss: 0.2995
Epoch 8/200
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8638 - loss: 0.2922 - val_accuracy: 0.8250 - 

## Revisar rendimientos

In [6]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9865 - loss: 0.0076 
Loss: 0.0074885254725813866, Accuracy: 0.9900000095367432


## Guardar modelo

In [ ]:
# Guardar el modelo si la precisión es adecuada

if accuracy > 0.9:
    model.save('../models/robot_arm_model.h5')
    print("Modelo guardado con éxito")
else:
    print("El modelo no tiene suficiente precisión")